In [1]:
import sys; sys.path.append("..") 
from src.utility import *
from src.dataprocessing import *
import numpy as np

# Data

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-4", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 200
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv'):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_200/dataset 200_1.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_2.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_3.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_4.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_5.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_6.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_7.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_8.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_9.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_10.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_11.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_12.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_13.csv
csv file is loaded from ./data/3_continuous/cts_200/dataset 200_14.csv
csv file is loa

## data to series

In [4]:
target_list = cts_list

history_length = 60
future_length = 20
step = 1

history_var = process_var
future_var = output_var

history_series = []
future_series = []

for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_length, history_var, future_length, future_var,
                                  step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

print(history_series.shape)
print(future_series.shape)

(29453, 60, 7)
(29453, 20, 3)


## supervised attention

In [5]:
from src import rnn
import matplotlib.pyplot as plt
'''
delta_list = [x for x in range(1,10)]
for delta in delta_list:
    linear_attention = rnn.super_attention(delta, future_length, 'linear')
    exp_attention = rnn.super_attention(delta, future_length, 'exp')
    
    print(f"delta {delta}")
    plt.figure(figsize=(12,6))
    
    plt.subplot(1,2,1)
    plt.plot([x for x in range(1,future_length+1)], linear_attention[0,:,0])
    plt.xlabel("time step")
    plt.xticks([x for x in range(1,future_length+1)])
    plt.ylabel("attention factor")
    
    plt.subplot(1,2,2)
    plt.plot([x for x in range(1,future_length+1)], exp_attention[0,:,0])
    plt.xlabel("time step")
    plt.xticks([x for x in range(1,future_length+1)])
    plt.ylabel("attention factor")
    
    plt.show()
'''

'\ndelta_list = [x for x in range(1,10)]\nfor delta in delta_list:\n    linear_attention = rnn.super_attention(delta, future_length, \'linear\')\n    exp_attention = rnn.super_attention(delta, future_length, \'exp\')\n    \n    print(f"delta {delta}")\n    plt.figure(figsize=(12,6))\n    \n    plt.subplot(1,2,1)\n    plt.plot([x for x in range(1,future_length+1)], linear_attention[0,:,0])\n    plt.xlabel("time step")\n    plt.xticks([x for x in range(1,future_length+1)])\n    plt.ylabel("attention factor")\n    \n    plt.subplot(1,2,2)\n    plt.plot([x for x in range(1,future_length+1)], exp_attention[0,:,0])\n    plt.xlabel("time step")\n    plt.xticks([x for x in range(1,future_length+1)])\n    plt.ylabel("attention factor")\n    \n    plt.show()\n'

## rnn model

In [6]:
history_series.shape

(29453, 60, 7)

In [7]:
from src import rnn

In [8]:
seq2seq_1 = rnn.RNN(history_series, future_series)
seq2seq_1.train_test(0.2, -1)

valid_size = seq2seq_1.history_test.shape[0]/seq2seq_1.history_train.shape[0]
seq2seq_1.train_valid(valid_size)

seq2seq_1.scaling()
seq2seq_1.build_model(2, 10, 'seq2seq_gru')
seq2seq_1.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 7)]      0           []                               
                                                                                                  
 encoder_1 (GRU)                (None, 60, 10)       570         ['input_1[0][0]']                
                                                                                                  
 encoder_2 (GRU)                [(None, 10),         660         ['encoder_1[0][0]']              
                                 (None, 10)]                                                      
                                                                                                  
 repeat_vector (RepeatVector)   (None, 20, 10)       0           ['encoder_2[0][0]']          

In [9]:
seq2seq_2 = rnn.RNN(history_series, future_series)
seq2seq_2.train_test(0.2, -1)

valid_size = seq2seq_2.history_test.shape[0]/seq2seq_2.history_train.shape[0]
seq2seq_2.train_valid(valid_size)

seq2seq_2.scaling()
seq2seq_2.build_model(3, 10, 'seq2seq_lstm')
seq2seq_2.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 60, 7)]      0           []                               
                                                                                                  
 encoder_1 (LSTM)               (None, 60, 10)       720         ['input_2[0][0]']                
                                                                                                  
 encoder_2 (LSTM)               (None, 60, 10)       840         ['encoder_1[0][0]']              
                                                                                                  
 encoder_3 (LSTM)               [(None, 10),         840         ['encoder_2[0][0]']              
                                 (None, 10),                                                

In [12]:
seq2seq_3 = rnn.RNN(history_series, future_series)
seq2seq_3.train_test(0.2, -1)

valid_size = seq2seq_3.history_test.shape[0]/seq2seq_3.history_train.shape[0]
seq2seq_3.train_valid(valid_size)

seq2seq_3.scaling()
seq2seq_3.build_model(3, 10, 'attn_seq2seq_lstm')
seq2seq_3.model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 60, 7)]      0           []                               
                                                                                                  
 encoder_1 (LSTM)               (None, 60, 10)       720         ['input_5[0][0]']                
                                                                                                  
 encoder_2 (LSTM)               (None, 60, 10)       840         ['encoder_1[0][0]']              
                                                                                                  
 encoder_3 (LSTM)               [(None, 60, 10),     840         ['encoder_2[0][0]']              
                                 (None, 10),                                                

In [13]:
seq2seq_4 = rnn.RNN(history_series, future_series)
seq2seq_4.train_test(0.2, -1)

valid_size = seq2seq_4.history_test.shape[0]/seq2seq_4.history_train.shape[0]
seq2seq_4.train_valid(valid_size)

seq2seq_4.scaling()
seq2seq_4.build_model(3, 10, 'attn_seq2seq_gru')
seq2seq_4.model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 60, 7)]      0           []                               
                                                                                                  
 encoder_1 (GRU)                (None, 60, 10)       570         ['input_6[0][0]']                
                                                                                                  
 encoder_2 (GRU)                (None, 60, 10)       660         ['encoder_1[0][0]']              
                                                                                                  
 encoder_3 (GRU)                [(None, 60, 10),     660         ['encoder_2[0][0]']              
                                 (None, 10)]                                                